In [6]:
from  ffmpy import FFmpeg as ffmpeg
import os
import re
import transfer
import asyncio
import redis
import smallLib as Helper
import asr_server as server

In [7]:
pool = redis.ConnectionPool(host='127.0.0.1',port=6379,db = 3,password='123456',decode_responses=True)   

permanent_table = 'all_user'
temp_table = "temp"
async def convert(farther_dir,tool):
    r = redis.Redis(connection_pool=pool)
    while r.hlen(temp_table)>0:
        name = r.get('curr')
        if name == None:
            user= Helper.get_user(r,temp_table)
            name = user[0]
            r.set('curr',name)
            if not os.path.exists(name+os.sep+tool):
                os.makedirs(name+os.sep+tool)
            sound_list = user[1].split(".wav,")
            for sound in sound_list:
                 r.hset(name,sound+".wav",0)
            
            #print(listx)
        while r.hlen(name) > 0:
            file  = Helper.redis_hpop(r,name)
            if file == None:
                continue
            input_path =name+os.sep+file
            out_path = name+os.sep+tool+os.sep+re.sub("(wav)","txt",file)
            #print(input_path+' '+out_path)
            try:
                
#                text = await(server.recognize('ws://localhost:2700',input_path))
                text =  await (transfer.run_test("ws://localhost:2700",input_path))
                with open (out_path, mode = 'w',encoding='utf-8') as f:
                    f.write(text)
                    r.hdel(name,file)
            except Exception:
                    print(out_path)
                    continue
        r.delete('curr',name)
        Helper.delete_user(r,temp_table,name)

In [8]:
from unittest.mock import patch


def init_table(farther_dir): 
    r = redis.Redis(connection_pool=pool)
    for listx in os.listdir(farther_dir):
        file_path = farther_dir+os.sep+listx
        path = ""
        for file in os.listdir(file_path):
            if file != "vosk":
                path+= file+","
        r.hset(temp_table ,file_path,path,)

#init_table("video")

In [9]:
#server.start()
import time 
import sys
def set_time(hour, minute):

    now = time.localtime(int(time.time()))
    a = str(str(now.tm_year)+'-'+str(now.tm_mon)+'-'+str(now.tm_mday)+' '+hour+':'+minute+':'+'00')
    targetTime = time.strptime(a,'%Y-%m-%d %H:%M:%S')
    return time.mktime(targetTime)

sys.argv=['','8','50']
targetTime = set_time(sys.argv[1],sys.argv[2])
print(targetTime)

while (time.time() < targetTime):
        time.sleep(5)


1663980600.0


In [10]:
await(convert("video","vosk"))

video\新疆逆龄努医生\vosk\.txt
video\新疆逆龄努医生\vosk\.txt
video\新疆逆龄努医生\vosk\.txt
video\新疆逆龄努医生\vosk\.txt
video\新疆逆龄努医生\vosk\.txt
video\新疆逆龄努医生\vosk\.txt
video\新疆逆龄努医生\vosk\.txt
video\新疆逆龄努医生\vosk\.txt
video\新疆逆龄努医生\vosk\.txt
